# Transfer learning 2

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data  import DataLoader,TensorDataset
import copy 
from sklearn.model_selection import train_test_split
import torchvision
import torchvision.transforms as T

import matplotlib.pyplot as plt
import pandas as pd

In [2]:
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[.485,.456,.406],std=[.229,.224,.225])
])

In [3]:
trainset = torchvision.datasets.STL10(root='../data',download=True,split='train',transform=transform)
testset = torchvision.datasets.STL10(root='../data',download=True,split='test',transform=transform)

100%|██████████| 2.64G/2.64G [15:04<00:00, 2.92MB/s]  


In [4]:
batch_size = 32
train_loader = DataLoader(trainset,batch_size=batch_size,shuffle=True,drop_last=True)
test_loader = DataLoader(testset,batch_size = 256)

In [7]:
trainset.data.shape , testset.data.shape

((5000, 3, 96, 96), (8000, 3, 96, 96))

# Import and inspect resnet-18 model

In [8]:
resent = torchvision.models.resnet18(pretrained=True)

/opt/homebrew/Caskroom/miniforge/base/envs/ia/lib/python3.13/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniforge/base/envs/ia/lib/python3.13/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /Users/mac/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:08<00:00, 5.49MB/s]


In [9]:
resent

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
device

device(type='mps')

In [13]:
from torchsummary import summary
summary(resent.to("cpu"),(3,96,96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 48, 48]           9,408
       BatchNorm2d-2           [-1, 64, 48, 48]             128
              ReLU-3           [-1, 64, 48, 48]               0
         MaxPool2d-4           [-1, 64, 24, 24]               0
            Conv2d-5           [-1, 64, 24, 24]          36,864
       BatchNorm2d-6           [-1, 64, 24, 24]             128
              ReLU-7           [-1, 64, 24, 24]               0
            Conv2d-8           [-1, 64, 24, 24]          36,864
       BatchNorm2d-9           [-1, 64, 24, 24]             128
             ReLU-10           [-1, 64, 24, 24]               0
       BasicBlock-11           [-1, 64, 24, 24]               0
           Conv2d-12           [-1, 64, 24, 24]          36,864
      BatchNorm2d-13           [-1, 64, 24, 24]             128
             ReLU-14           [-1, 64,

In [14]:
# freez all the layers
for p in resent.parameters():
    p.requires_grad = False

In [15]:
resent.fc = nn.Linear(512,10)
resent.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [20]:
lossfun = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(resent.parameters(),lr=.001,momentum=.9)

In [21]:
import sys
sys.path.append('../utils')
sys.path.append('../models')
import importlib 
import training
importlib.reload(training)
from training import trainTheM0del

In [23]:
losses, train_accuracy, test_accuracy = trainTheM0del(
        isClassification=True,
        optimizer = optimizer,
        doBN=False,
        model=resent,
        train_loader=train_loader,
        test_loader=test_loader,
        num_epochs=20, 
        loss_function= lossfun,
        device = device
        )

Epoch 1/20, Train Loss: 0.5146, Train Acc: 82.63, Test Acc: 77.34
Epoch 2/20, Train Loss: 0.5100, Train Acc: 82.77, Test Acc: 76.56
Epoch 3/20, Train Loss: 0.4940, Train Acc: 83.47, Test Acc: 77.73
Epoch 4/20, Train Loss: 0.4800, Train Acc: 83.57, Test Acc: 76.56
Epoch 5/20, Train Loss: 0.4930, Train Acc: 82.99, Test Acc: 79.69
Epoch 6/20, Train Loss: 0.4831, Train Acc: 83.57, Test Acc: 76.56
Epoch 7/20, Train Loss: 0.4828, Train Acc: 83.17, Test Acc: 76.17
Epoch 8/20, Train Loss: 0.4599, Train Acc: 84.23, Test Acc: 78.91
Epoch 9/20, Train Loss: 0.4599, Train Acc: 83.99, Test Acc: 76.95
Epoch 10/20, Train Loss: 0.4683, Train Acc: 83.59, Test Acc: 76.56
Epoch 11/20, Train Loss: 0.4612, Train Acc: 84.09, Test Acc: 77.34
Epoch 12/20, Train Loss: 0.4590, Train Acc: 84.05, Test Acc: 76.56
Epoch 13/20, Train Loss: 0.4603, Train Acc: 84.03, Test Acc: 77.34
Epoch 14/20, Train Loss: 0.4563, Train Acc: 84.66, Test Acc: 76.95
Epoch 15/20, Train Loss: 0.4445, Train Acc: 84.42, Test Acc: 78.91
Epoc